## RAG Pipelinefor Data ingetion to Vector DB

In [16]:
import os
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [17]:
### Read Doc

def process_documents(file_path):
 
    all_pages = []
    DIR = Path(file_path)

    doc_files=list(DIR.glob("**/*.doc"))
    print("Document Files Length: ", len(doc_files))
    for file_path in doc_files:
        try:
            loader = PyMuPDFLoader(str(file_path))
            document = loader.load()
            # print(documents)
            for pages in document:
                pages.metadata['source'] = str(file_path.name)
                pages.metadata['file_type'] = 'doc'
            all_pages.extend(document)
            print(f"Loaded {file_path.name}, total pages: {len(document)}")
        except:
            print(f"Error loading {file_path.name}, skipping.")
            continue
    # print("Total Documents Loaded: ", len(all_pages))
    return all_pages


all_subren_pages = process_documents("../data/SUBREN")


Document Files Length:  4492
Loaded SUBREN-4445.doc, total pages: 5
Loaded SUBREN-2034.doc, total pages: 6
Loaded SUBREN-2752.doc, total pages: 19
Loaded SUBREN-4323.doc, total pages: 5
Loaded SUBREN-2746.doc, total pages: 31
Loaded SUBREN-4337.doc, total pages: 6
Loaded SUBREN-3458.doc, total pages: 4
Loaded SUBREN-4451.doc, total pages: 3
Loaded SUBREN-1529.doc, total pages: 6
Loaded SUBREN-808.doc, total pages: 23
Loaded SUBREN-3316.doc, total pages: 8
Loaded SUBREN-4479.doc, total pages: 5
Loaded SUBREN-2008.doc, total pages: 4
Loaded SUBREN-1501.doc, total pages: 4
Loaded SUBREN-820.doc, total pages: 6
Loaded SUBREN-1267.doc, total pages: 4
Loaded SUBREN-3470.doc, total pages: 6
Loaded SUBREN-3464.doc, total pages: 3
Loaded SUBREN-1273.doc, total pages: 10
Loaded SUBREN-1515.doc, total pages: 3
Loaded SUBREN-834.doc, total pages: 3
Loaded SUBREN-3302.doc, total pages: 26
Loaded SUBREN-4486.doc, total pages: 4
Loaded SUBREN-607.doc, total pages: 3
Loaded SUBREN-1298.doc, total page

In [18]:
all_subren_pages

[Document(metadata={'source': 'SUBREN-4445.doc', 'file_path': '../data/SUBREN/SUBREN-4445.doc', 'page': 0, 'total_pages': 5, 'format': 'HTML5', 'title': '[#SUBREN-4445] Production Bug : CFD prints Adjustment even though price adjustment has not been applied', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': '', 'encryption': '', 'file_type': 'doc'}, page_content='[SUBREN-4445]\xa0Production Bug : CFD\nprints Adjustment even though price\nadjustment has not been applied Created:\n19/Nov/24 \xa0Updated: 17/Dec/24\nStatus:\nTo Do\nProject:\nSubscription &\nRenewals\nComponent/s:\nNone\nAffects Version/s:\nNone\nFix Version/s:\nNone\nType:\nBug\nPriority:\nNone\nReporter:\nNamrata\nPagaria\nAssignee: Sumit\nKapoor\nResolution:\nUnresolved\nVotes:\n0\nLabels:\nNone\nRemaining\nEstimate:\nNot Specified\nTime Spent:\nNot Specified\nOriginal Estimate:\nNot Specified\nManage\nCustomers\nButton:\nManage Customers\n•  Add cu

### Text Splitting

In [19]:
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs= text_splitter.split_documents(documents)
    print(f"Split {len(documents)} Documents into ", len(split_docs), " chunks.")

    if split_docs:
        print("example chunk : ")
        print("Sample Content: ", split_docs[0].page_content[:300], "...")
        print("Metadata: ", split_docs[0].metadata)

    return split_docs

all_subren_docs = split_documents(all_subren_pages)

Split 29020 Documents into  34950  chunks.
example chunk : 
Sample Content:  [SUBREN-4445] Production Bug : CFD
prints Adjustment even though price
adjustment has not been applied Created:
19/Nov/24  Updated: 17/Dec/24
Status:
To Do
Project:
Subscription &
Renewals
Component/s:
None
Affects Version/s:
None
Fix Version/s:
None
Type:
Bug
Priority:
None
Reporter:
Namrata
Pagari ...
Metadata:  {'source': 'SUBREN-4445.doc', 'file_path': '../data/SUBREN/SUBREN-4445.doc', 'page': 0, 'total_pages': 5, 'format': 'HTML5', 'title': '[#SUBREN-4445] Production Bug : CFD prints Adjustment even though price adjustment has not been applied', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': '', 'encryption': '', 'file_type': 'doc'}


In [20]:
all_subren_docs

[Document(metadata={'source': 'SUBREN-4445.doc', 'file_path': '../data/SUBREN/SUBREN-4445.doc', 'page': 0, 'total_pages': 5, 'format': 'HTML5', 'title': '[#SUBREN-4445] Production Bug : CFD prints Adjustment even though price adjustment has not been applied', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': '', 'encryption': '', 'file_type': 'doc'}, page_content='[SUBREN-4445]\xa0Production Bug : CFD\nprints Adjustment even though price\nadjustment has not been applied Created:\n19/Nov/24 \xa0Updated: 17/Dec/24\nStatus:\nTo Do\nProject:\nSubscription &\nRenewals\nComponent/s:\nNone\nAffects Version/s:\nNone\nFix Version/s:\nNone\nType:\nBug\nPriority:\nNone\nReporter:\nNamrata\nPagaria\nAssignee: Sumit\nKapoor\nResolution:\nUnresolved\nVotes:\n0\nLabels:\nNone\nRemaining\nEstimate:\nNot Specified\nTime Spent:\nNot Specified\nOriginal Estimate:\nNot Specified\nManage\nCustomers\nButton:\nManage Customers\n•  Add cu

### Embedding & VectorStore DB

In [21]:
import chromadb
from chromadb.config import Settings
import numpy as np
from sentence_transformers import SentenceTransformer
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [28]:
class EmbeddingManager:
    def __init__(self,model_name : str ="all-MiniLM-L6-v2"):
        self.model_name = model_name #This model has 384 dimensions
        self.model= None
        self._load_model()

    def _load_model(self):
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully: {self.model.get_sentence_embedding_dimension()} dimensions")
        except Exception as e:
            print(f"Error loading model: {e}")

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Embedding model is not loaded.")
        embeddings = self.model.encode(texts, show_progress_bar=True, convert_to_numpy=True)
        print(f"Generated embeddings for {len(texts)} texts.")
        print(f"Embedding shape: {embeddings.shape}")
        return embeddings

In [27]:
d= EmbeddingManager()
d.model


Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully: 384 dimensions


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

### Vectore Store

In [ ]:
class VectorStore:
    def __init__(self, collection_name: str = "Doc_data", persist_directory: str = "./data/vector_store"):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self.initialize_store()
 

    def _initialize_store(self):
        try:
            print("Initializing ChromaDB client...")
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.Client(Settings(
                chroma_db_impl="duckdb+parquet",
                persist_directory=self.persist_directory
            ))
            print("ChromaDB client initialized.")


            self.collection = self.client.get_or_create_collection(name=self.collection_name, metadata={"Description": "DOC embedding for RAG."})
            print(f"Collection '{self.collection_name}' is ready.")
            print("Existing Documents in collection: ", self.collection.count())
        except Exception as e:
            print(f"Error initializing vector store: {e}")
    
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents and embeddings must match.")
        
        print(f"Adding {len(documents)} documents to the vector store...")

        ids=[]
        metadatas=[]
        documents_texts=[]
        embeddings_list=[]

        for i,(doc, emb) in enumerate(zip(documents, embeddings)):
            # Generate a unique ID for each document
            doc_id=f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            #Make metadata
            metadata=dict(doc.metadata)
            metadata['doc_index']= i
            metadata['content_length']= len(doc.page_content)
            metadatas.append(metadata)

            # Document Context
            documents_texts.append(doc.page_content)

            # Embeddings
            embeddings_list.append(emb.tolist())
        

In [30]:
v= VectorStore()
v.collection

AttributeError: 'VectorStore' object has no attribute 'initialize_store'